# Introduction

In [3]:
from collections import OrderedDict
import functools

import math
import torch
from torch.distributions import constraints

import funsor
from funsor.terms import Funsor, Variable
from funsor.tensor import Tensor
from funsor.domains import Array, Bint, Real, Reals
from funsor.factory import Bound, Fresh, Has, Value, make_funsor, to_funsor
import funsor.ops as ops
from funsor.cnf import Contraction
from funsor.testing import random_tensor

funsor.set_backend("torch")
torch.set_default_dtype(torch.float32)

%load_ext mypy_ipython

The mypy_ipython extension is already loaded. To reload it, use:
  %reload_ext mypy_ipython


# Informal Overview

## Named Tensors

In [4]:
A = Tensor(
    torch.tensor([[3., 1., 4.],
                  [1., 5., 9.],
                  [2., 6., 5.]]),
    inputs=OrderedDict([("height", Bint[3]), ("width", Bint[3])]),
    dtype="real"
)

In [5]:
A(height=0)

Tensor(tensor([3., 1., 4.]), OrderedDict([('width', Bint[3, ])]), 'real')

In [6]:
A(width=2)

Tensor(tensor([4., 9., 5.]), OrderedDict([('height', Bint[3, ])]), 'real')

## Named tensor operations

### Elementwise operations and broadcasting

In [7]:
A.sigmoid()  # 1 / (1 + (-A).exp())

Tensor(tensor([[0.9526, 0.7311, 0.9820],
        [0.7311, 0.9933, 0.9999],
        [0.8808, 0.9975, 0.9933]]), OrderedDict([('height', Bint[3, ]), ('width', Bint[3, ])]), 'real')

In [8]:
x = Tensor(
    torch.tensor([2., 7., 1.]),
    inputs=OrderedDict([("height", Bint[3])]),
    dtype="real"
)

y = Tensor(
    torch.tensor([1., 4., 1.]),
    inputs=OrderedDict([("width", Bint[3])]),
    dtype="real"
)

In [9]:
A + x

Tensor(tensor([[ 5.,  3.,  6.],
        [ 8., 12., 16.],
        [ 3.,  7.,  6.]]), OrderedDict([('height', Bint[3, ]), ('width', Bint[3, ])]), 'real')

In [10]:
A + y

Tensor(tensor([[ 4.,  5.,  5.],
        [ 2.,  9., 10.],
        [ 3., 10.,  6.]]), OrderedDict([('height', Bint[3, ]), ('width', Bint[3, ])]), 'real')

### Reductions

In [11]:
A.reduce(ops.add, reduced_vars="height")

Tensor(tensor([ 6., 12., 18.]), OrderedDict([('width', Bint[3, ])]), 'real')

In [12]:
A.reduce(ops.add, reduced_vars="width")

Tensor(tensor([ 8., 15., 13.]), OrderedDict([('height', Bint[3, ])]), 'real')

In [13]:
A.reduce(ops.add, reduced_vars={"height", "width"})

Tensor(36.0, OrderedDict(), 'real')

In [14]:
(A * y).reduce(ops.add, reduced_vars="width")

Tensor(tensor([11., 30., 31.]), OrderedDict([('height', Bint[3, ])]), 'real')

In [15]:
Contraction(
    red_op=ops.add,
    bin_op=ops.mul,
    reduced_vars=frozenset({Variable("width", output=Bint[3])}),
    terms=(A, y)
)

Tensor(tensor([11., 30., 31.]), OrderedDict([('height', Bint[3, ])]), 'real')

In [16]:
(x * x).reduce(ops.add, reduced_vars="height")  # innder product
x * y  # outer product
(A * y).reduce(ops.add, reduced_vars="width")  # matrix-vector product
# vector-matrix product is the same as matrix-vector product
(A * x).reduce(ops.add, reduced_vars="height")  # vector-matrix product

Tensor(tensor([15., 43., 76.]), OrderedDict([('width', Bint[3, ])]), 'real')

In [17]:
B = Tensor(
    torch.tensor([[3, 2, 5], [5, 4, 0], [8, 3, 6]]),
    inputs=OrderedDict([("width", Bint[3]), ("width'", Bint[3])]),
    dtype="real"
)

In [18]:
(A * B).reduce(ops.add, reduced_vars="width")  # matrix-matrix product

Tensor(tensor([[ 46.,  22.,  39.],
        [100.,  49.,  59.],
        [ 76.,  43.,  40.]]), OrderedDict([('height', Bint[3, ]), ("width'", Bint[3, ])]), 'real')

### Renaming and reshaping

In [19]:
A(**{"height": "height'"})  # A(height="height'")

Tensor(tensor([[3., 1., 4.],
        [1., 5., 9.],
        [2., 6., 5.]]), OrderedDict([("height'", Bint[3, ]), ('width', Bint[3, ])]), 'real')

# Examples

## Building blocks

### Feedforward neural networks

In [20]:
input_dim = 100
X0 = random_tensor(
    OrderedDict([("input_layer", Bint[input_dim])])
)

hidden_1_dim = 32
W1 = random_tensor(
    OrderedDict([
        ("input_layer", Bint[input_dim]),
        ("hidden_layer_1", Bint[hidden_1_dim])
    ])
)
b1 = random_tensor(
    OrderedDict([("hidden_layer_1", Bint[hidden_1_dim])])
)
X1 = ((W1 * X0).reduce(ops.add, "input_layer") + b1).sigmoid()

hidden_2_dim = 16
W2 = random_tensor(
    OrderedDict([
        ("hidden_layer_1", Bint[hidden_1_dim]),
        ("hidden_layer_2", Bint[hidden_2_dim])
    ])
)
b2 = random_tensor(
    OrderedDict([("hidden_layer_2", Bint[hidden_2_dim])])
)
X2 = ((W2 * X1).reduce(ops.add, "hidden_layer_1") + b2).sigmoid()

hidden_3_dim = 8
W3 = random_tensor(
    OrderedDict([
        ("hidden_layer_2", Bint[hidden_2_dim]),
        ("hidden_layer_3", Bint[hidden_3_dim])
    ])
)
b3 = random_tensor(
    OrderedDict([("hidden_layer_3", Bint[hidden_3_dim])])
)
X3 = ((W3 * X2).reduce(ops.add, "hidden_layer_2") + b3).sigmoid()

In [24]:
%mypy

# is this correct?
@make_funsor
def FullConnLayer(
    x: Has[{"layer"}],
    W: Has[{"layer", "out_layer"}],
    b: Has[{"out_layer"}],
    out_layer: Bound
) -> Fresh[lambda x: x]:
    result = ((W * x).reduce(ops.add, "layer") + b).sigmoid()
    return result(out_layer="layer")

Success: no issues found in 1 source file
Type checking successful


In [25]:
input_dim = 100
X0 = random_tensor(
    OrderedDict([("layer", Bint[input_dim])])
)
hidden_1_dim = 32
W1 = random_tensor(
    OrderedDict([
        ("layer", Bint[input_dim]),
        ("out_layer", Bint[hidden_1_dim])
    ])
)
b1 = random_tensor(
    OrderedDict([("out_layer", Bint[hidden_1_dim])])
)

X1 = FullConnLayer(X0, W1, b1, "out_layer")
X1

Tensor(tensor([5.1424e-05, 1.6135e-04, 6.2248e-01, 9.9984e-01, 9.9932e-01, 2.8701e-01,
        1.0000e+00, 1.1595e-01, 1.0000e+00, 3.0341e-05, 1.2613e-07, 4.2051e-05,
        1.1197e-06, 9.9520e-01, 9.9988e-01, 3.3333e-02, 2.7272e-01, 5.8579e-04,
        7.3940e-04, 9.9880e-01, 4.9861e-01, 9.9999e-01, 9.8301e-01, 8.3890e-02,
        2.2161e-04, 7.4274e-01, 1.0000e+00, 9.9943e-01, 8.3586e-01, 9.8640e-01,
        1.0526e-12, 7.3241e-01]), OrderedDict([('layer', Bint[32, ])]), 'real')

### Recurrent neural networks

In [ ]:
@make_funsor
def RecurrentLayer(
    x: Funsor,
    Wh: Funsor,
    Wi: Funsor,
    b: Funsor,
    hidden: Bound,
    input: Bound
) -> Fresh[lambda x: x]:
    output = ((Wh * h).reduce(ops.add, "hidden") + (Wi * x).reduce(ops.add, "input") + b).sigmoid()
    return output(hidden="new_hidden")

### Attention

In [26]:
# Can Has be used here?
@make_funsor
def Softmax(
    x: Funsor,
    i: Funsor
) -> Fresh[lambda x: x]:
    return x.exp() / x.exp().reduce(ops.add, i)

In [27]:
@make_funsor
def Attention(
    Q: Has[{"key"}],
    K: Has[{"key"}],
    V: Has[{"seq"}],
    M: Has[{"seq"}], # does only one of Q, K, and M need to have "seq"?
    key: Bound,
    seq: Bound
) -> Fresh[lambda Q: Q]:
    x = (Q * K).reduce(ops.add, key) / math.sqrt(key.output.size) + M
    return (Softmax(x, seq) * V).reduce(ops.add, seq)

In [28]:
q = random_tensor(OrderedDict([("key", Bint[10])]))
k = random_tensor(OrderedDict([("key", Bint[10]), ("seq", Bint[3])]))
v = random_tensor(OrderedDict([("seq", Bint[3]), ("val", Bint[5])]))
m = random_tensor(OrderedDict([("seq", Bint[3])]))
Attention(q, k, v, m, "key", "seq")

Tensor(tensor([-1.1402,  0.1616, -0.3963, -1.2688,  0.0687]), OrderedDict([('val', Bint[5, ])]), 'real')

### Convolution

In [152]:
@make_funsor
def Unroll(
    x: Funsor,
    seq: Bound,
    new_seq: Fresh[lambda seq: Bint[seq.size - 1]],
    kernel: Fresh[lambda: Bint[2]],
) -> Fresh[lambda x: x]:
    return x(**{seq.name: new_seq+kernel-1})

In [156]:
q = random_tensor(OrderedDict([("seq", Bint[10])]))
# Unroll(q, "seq", "new_seq", "kernel")
new_seq = Variable("new_seq", Bint[9])
kernel = Variable("kernel", Bint[2])
q(seq=new_seq+kernel)

Tensor(tensor([[-2.4158, -0.3009],
        [-0.3009, -0.0916],
        [-0.0916,  0.3367],
        [ 0.3367, -0.2952],
        [-0.2952, -0.2060],
        [-0.2060, -0.8539],
        [-0.8539,  1.2039],
        [ 1.2039, -1.1123],
        [-1.1123, -1.9163]]), OrderedDict([('new_seq', Bint[9, ]), ('kernel', Bint[2, ])]), 'real')

In [157]:
new_seq + kernel

Variable('new_seq', Bint[9, ]) + Variable('kernel', Bint[2, ]).reduce(nullop, set())

In [ ]:
# v(seq=2, kernel=1) == q(seq=2+1-1)

### Max pooling

### Normalization layers

In [61]:
@make_funsor
def Mean(
    X: Funsor,
    ax: Bound
) -> Fresh[lambda X: X]:
    return X.reduce(ops.add, ax) / ax.output.size

@make_funsor
def Variance(
    X: Funsor,
    ax: Bound
) -> Fresh[lambda X: X]:
    return Mean((X - Mean(X, ax))**2, ax)

@make_funsor
def Standardize(
    X: Funsor,
    ax: Bound
) -> Fresh[lambda X: X]:
    return (X - Mean(X, ax)) / (Variance(X, ax) + ops.finfo(X.data).eps)

@make_funsor
def BatchNorm(
    X: Funsor,
    gamma: Funsor,
    beta: Funsor,
    batch: Bound,
    layer: Bound
) -> Fresh[lambda X: X]:
    return Standardize(X, )

In [62]:
x = random_tensor(OrderedDict([("i", Bint[10])]))
Standardize(x, "i")

Tensor(tensor([-2.0574, -0.4460,  1.0813,  0.4329,  0.9293, -0.0551,  0.2744, -0.5563,
        -1.1598,  1.5569]), OrderedDict([('i', Bint[10, ])]), 'real')

## Transformer

## LeNet

## Other examples

### Discrete random variables

In [29]:
B_given_A = Tensor(
    torch.tensor([[0.2, 0.3, 0.5],
                  [0.8, 0.1, 0.1]]),
    inputs=OrderedDict([("A", Bint[2]), ("B", Bint[3])]),
    dtype="real"
)

A = Tensor(
    torch.tensor([0.6, 0.4]),
    inputs=OrderedDict([("A", Bint[2])]),
    dtype="real"
)

In [30]:
A_and_B = B_given_A * A
A_and_B

Tensor(tensor([[0.1200, 0.1800, 0.3000],
        [0.3200, 0.0400, 0.0400]]), OrderedDict([('A', Bint[2, ]), ('B', Bint[3, ])]), 'real')

In [31]:
B = A_and_B.reduce(ops.add, "A")
B

Tensor(tensor([0.4400, 0.2200, 0.3400]), OrderedDict([('B', Bint[3, ])]), 'real')

In [32]:
A_given_B = A_and_B / B
A_given_B

Tensor(tensor([[0.2727, 0.8182, 0.8824],
        [0.7273, 0.1818, 0.1176]]), OrderedDict([('A', Bint[2, ]), ('B', Bint[3, ])]), 'real')

### Advanced indexing

### Continuous bag of words

### Sudoku ILP

### K-means clustering

In [ ]:
x = random_tensor(OrderedDict([("batch", Bint[10]), ("d", Bint[4])]))
c = random_tensor(OrderedDict([("clusters", Bint[3]), ("d", Bint[4])]))

### Beam search

### Multivariate normal distribution

# LaTeX Macros

# Formal Definitions

## Records and shapes

## Named tensors

## Named tensor operations

## Common operations

# Differentiation

## Definition

## Rules

## Example

## Broadcasting

# Alternatives